In [145]:
#全データに対応できる汎用的なモデルを作成する
#深層モジュラーファジィ推論モデル
import numpy as np
import pandas as pd
import random
import math
import sys
import time
from scipy import special

In [146]:
#各種データの読み込みを行い、正規化を行う
#ただし、正規化したくないカラム名は例外カラム欄に格納しておく
original_data = pd.read_csv('fuzzy_diabetes2_1.csv')

# exception_columns = []
# for clm in original_data.columns:
#     if clm not in exception_columns:
#         original_data[clm] = (original_data[clm] - original_data[clm].min()) / (original_data[clm].max() - original_data[clm].min())
np_original_data = np.array(original_data)

In [147]:
#元データの入力データと出力データの合計の属性数を求める
#また入力変数の属性数を求めるが、ここでは出力変数の属性数が1であることを仮定している
NUMBER_OF_INPUT_OUTPUT = len(np_original_data[0,:])
NUMBER_OF_INPUT = NUMBER_OF_INPUT_OUTPUT - 1
NUMBER_OF_INPUT_LAYER1 = NUMBER_OF_INPUT
#2層目以降は前層までの出力を現層の入力として扱う
NUMBER_OF_INPUT_LAYER2 = NUMBER_OF_INPUT_LAYER1 + 1
NUMBER_OF_INPUT_LAYER3 = NUMBER_OF_INPUT_LAYER2 + 1

#またファジィ分割数と各層でのモジュラー数、各層の各モジュラー内でのファジィルール数を求める
NUMBER_OF_FUZZY_PARTITION = 3
NUMBER_OF_MODULAR_LAYER1 = special.comb(NUMBER_OF_INPUT, 2, True)
NUMBER_OF_MODULAR_LAYER2 = special.comb(NUMBER_OF_INPUT+1, 2, True)
NUMBER_OF_MODULAR_LAYER3 = special.comb(NUMBER_OF_INPUT+2, 2, True)
NUMBER_OF_FUZZY_RULE_OF_MODULAR = NUMBER_OF_FUZZY_PARTITION ** 2

In [148]:
#1層目における教師データとテストデータを作成する
TEACHER_SIZE_RATIO = 5
TEST_SIZE_RATIO = 5
DATA_SIZE = len(np_original_data)
TEACHER_SIZE = ((int)(len(np_original_data) * ((TEACHER_SIZE_RATIO) / (TEACHER_SIZE_RATIO + TEST_SIZE_RATIO)))) + 1
TEST_SIZE = len(np_original_data) - TEACHER_SIZE

teacher_input_layer1 = np_original_data[:TEACHER_SIZE, :NUMBER_OF_INPUT]
teacher_output_layer1 = np_original_data[:TEACHER_SIZE, NUMBER_OF_INPUT]

In [149]:
#1~3層において用いる前件部と後件部の格納先を作成する
antecedent_center_layer1 = np.empty((NUMBER_OF_MODULAR_LAYER1, NUMBER_OF_FUZZY_RULE_OF_MODULAR, 2))
antecedent_broad_layer1 = np.empty((NUMBER_OF_MODULAR_LAYER1, NUMBER_OF_FUZZY_RULE_OF_MODULAR, 2))
antecedent_center_layer2 = np.empty((NUMBER_OF_MODULAR_LAYER2, NUMBER_OF_FUZZY_RULE_OF_MODULAR, 2))
antecedent_broad_layer2 = np.empty((NUMBER_OF_MODULAR_LAYER2, NUMBER_OF_FUZZY_RULE_OF_MODULAR, 2))
antecedent_center_layer3 = np.empty((NUMBER_OF_MODULAR_LAYER3, NUMBER_OF_FUZZY_RULE_OF_MODULAR, 2))
antecedent_broad_layer3 = np.empty((NUMBER_OF_MODULAR_LAYER3, NUMBER_OF_FUZZY_RULE_OF_MODULAR, 2))

consequent_layer1 = np.empty((NUMBER_OF_MODULAR_LAYER1, NUMBER_OF_FUZZY_RULE_OF_MODULAR))
consequent_part_layer2 = np.empty((NUMBER_OF_MODULAR_LAYER2, NUMBER_OF_FUZZY_RULE_OF_MODULAR, 2))
consequent_layer2 = np.empty((NUMBER_OF_MODULAR_LAYER2, NUMBER_OF_FUZZY_RULE_OF_MODULAR))
consequent_part_layer3 = np.empty((NUMBER_OF_MODULAR_LAYER3, NUMBER_OF_FUZZY_RULE_OF_MODULAR, 3))
consequent_layer3 = np.empty((NUMBER_OF_MODULAR_LAYER3, NUMBER_OF_FUZZY_RULE_OF_MODULAR))

modular_weight_layer1 = np.empty(NUMBER_OF_MODULAR_LAYER1)
modular_weight_layer2 = np.empty(NUMBER_OF_MODULAR_LAYER2)
modular_weight_layer3 = np.empty(NUMBER_OF_MODULAR_LAYER3)

In [150]:
#1~3層において用いる前件部と後件部、モジュラーの重みの初期化を行う
#モジュラーの重みの初期設定する
for i in range(NUMBER_OF_MODULAR_LAYER1):
    modular_weight_layer1[i] = 0.1
for i in range(NUMBER_OF_MODULAR_LAYER2):
    modular_weight_layer2[i] = 0.07
for i in range(NUMBER_OF_MODULAR_LAYER3):
    modular_weight_layer3[i] = 0.05
#まずは後件部実数値の初期設定を行う
for i in range(NUMBER_OF_MODULAR_LAYER1):
    for j in range(NUMBER_OF_FUZZY_RULE_OF_MODULAR):
        consequent_layer1[i,j] = 0.3
for i in range(NUMBER_OF_MODULAR_LAYER2):
    for j in range(NUMBER_OF_FUZZY_RULE_OF_MODULAR):
        for k in range(2):
            consequent_part_layer2[i,j,k] = 0.3
for i in range(NUMBER_OF_MODULAR_LAYER3):
    for j in range(NUMBER_OF_FUZZY_RULE_OF_MODULAR):
        for k in range(3):
            consequent_part_layer3[i,j,k] = 0.3
#前件部を初期設定する
for i in range(NUMBER_OF_MODULAR_LAYER1):
    for j in range(NUMBER_OF_FUZZY_RULE_OF_MODULAR):
        for k in range(2):
            if k == 0:
                if (int)(j / 3) == 0:
                    antecedent_center_layer1[i,j,k] = 0
                    antecedent_broad_layer1[i,j,k] = 1
                elif (int)(j / 3) == 1:
                    antecedent_center_layer1[i,j,k] = 0.5
                    antecedent_broad_layer1[i,j,k] = 0.5
                else:
                    antecedent_center_layer1[i,j,k] = 1
                    antecedent_broad_layer1[i,j,k] = 1
            else:
                if j % 3 == 0:
                    antecedent_center_layer1[i,j,k] = 0
                    antecedent_broad_layer1[i,j,k] = 1
                elif j % 3 == 1:
                    antecedent_center_layer1[i,j,k] = 0.5
                    antecedent_broad_layer1[i,j,k] = 0.5
                else:
                    antecedent_center_layer1[i,j,k] = 1
                    antecedent_broad_layer1[i,j,k] = 1
for i in range(NUMBER_OF_MODULAR_LAYER2):
    for j in range(NUMBER_OF_FUZZY_RULE_OF_MODULAR):
        for k in range(2):
            if k == 0:
                if (int)(j / 3) == 0:
                    antecedent_center_layer2[i,j,k] = 0
                    antecedent_broad_layer2[i,j,k] = 1
                elif (int)(j / 3) == 1:
                    antecedent_center_layer2[i,j,k] = 0.5
                    antecedent_broad_layer2[i,j,k] = 0.5
                else:
                    antecedent_center_layer2[i,j,k] = 1
                    antecedent_broad_layer2[i,j,k] = 1
            else:
                if j % 3 == 0:
                    antecedent_center_layer2[i,j,k] = 0
                    antecedent_broad_layer2[i,j,k] = 1
                elif j % 3 == 1:
                    antecedent_center_layer2[i,j,k] = 0.5
                    antecedent_broad_layer2[i,j,k] = 0.5
                else:
                    antecedent_center_layer2[i,j,k] = 1
                    antecedent_broad_layer2[i,j,k] = 1
for i in range(NUMBER_OF_MODULAR_LAYER3):
    for j in range(NUMBER_OF_FUZZY_RULE_OF_MODULAR):
        for k in range(2):
            if k == 0:
                if (int)(j / 3) == 0:
                    antecedent_center_layer3[i,j,k] = 0
                    antecedent_broad_layer3[i,j,k] = 1
                elif (int)(j / 3) == 1:
                    antecedent_center_layer3[i,j,k] = 0.5
                    antecedent_broad_layer3[i,j,k] = 0.5
                else:
                    antecedent_center_layer3[i,j,k] = 1
                    antecedent_broad_layer3[i,j,k] = 1
            else:
                if j % 3 == 0:
                    antecedent_center_layer3[i,j,k] = 0
                    antecedent_broad_layer3[i,j,k] = 1
                elif j % 3 == 1:
                    antecedent_center_layer3[i,j,k] = 0.5
                    antecedent_broad_layer3[i,j,k] = 0.5
                else:
                    antecedent_center_layer3[i,j,k] = 1
                    antecedent_broad_layer3[i,j,k] = 1

In [151]:
%%time
#1層目の学習を行う
#学習回数、学習係数を設定する
TRAIN_TIME = 1000
LEARNING_CONSEQUENT = 0.0001
LEARNING_WEIGHT = 0.0001

#平均二条誤差を設定する
mean_squared_error_layer1 = 0

#適合度・メンバシップ関数値を初期化する
adaptability = np.ones((TEACHER_SIZE, NUMBER_OF_MODULAR_LAYER1, NUMBER_OF_FUZZY_RULE_OF_MODULAR))
membership_function = np.ones((TEACHER_SIZE, NUMBER_OF_MODULAR_LAYER1, NUMBER_OF_FUZZY_RULE_OF_MODULAR, 2))

for i in range(TEACHER_SIZE):
    for j in range(NUMBER_OF_MODULAR_LAYER1):
        for k in range(NUMBER_OF_FUZZY_RULE_OF_MODULAR):
            for l in range(2):
                #次にモジュラーの通し番号から用いる入力変数の属性を求める
                if l == 0:
                    #1つ目の入力変数属性に対して
                    count = 1
                    a = j
                    while True:
                        a = a - (NUMBER_OF_INPUT_LAYER1 - count)
                        count = count + 1
                        if a < 0:
                            m = count - 2
                            break
                elif l == 1:
                    #2つ目の入力変数に対して
                    count = 1
                    a = j + 1
                    while True:
                        a = a - (NUMBER_OF_INPUT_LAYER1 - count)
                        count = count + 1
                        if a <= 0:
                            count = count - 1
                            n = NUMBER_OF_INPUT - 1
                            m = n + a
                            break
                if(teacher_input_layer1[i,m] >= antecedent_center_layer1[j,k,l] - antecedent_broad_layer1[j,k,l]) and (teacher_input_layer1[i,m] <= antecedent_center_layer1[j,k,l]):
                    membership_function[i,j,k,l] = (teacher_input_layer1[i,m] - (antecedent_center_layer1[j,k,l] - antecedent_broad_layer1[j,k,l])) / antecedent_broad_layer1[j,k,l]
                elif(teacher_input_layer1[i,m] > antecedent_center_layer1[j,k,l]) and (teacher_input_layer1[i,m] <= antecedent_center_layer1[j,k,l] + antecedent_broad_layer1[j,k,l]):
                    membership_function[i,j,k,l] = -(teacher_input_layer1[i,m] - (antecedent_center_layer1[j,k,l] + antecedent_broad_layer1[j,k,l])) / antecedent_broad_layer1[j,k,l]
            #適合度を求める
            for l in range(2):
                adaptability[i,j,k] = adaptability[i,j,k] * membership_function[i,j,k,l]
#学習を行う
for time in range(TRAIN_TIME):
    #各データの予測結果を格納する
    output_layer1 = np.empty(TEACHER_SIZE)
    #各モジュールごとの出力値を格納する
    output_modular_layer1 = np.empty((TEACHER_SIZE, NUMBER_OF_MODULAR_LAYER1))
    for i in range(TEACHER_SIZE):
        for j in range(NUMBER_OF_MODULAR_LAYER1):
            #各データのモジュールごとの予測結果を求める
            output_modular_layer1[i,j] = np.sum(np.dot(adaptability[i,j], consequent_layer1[j])) / np.sum(adaptability[i,j])
        #各データの予測結果を求める
        output_layer1[i] = np.sum(np.dot(output_modular_layer1[i], modular_weight_layer1))
        if(time == TRAIN_TIME-1):
            mean_squared_error_layer1 = mean_squared_error_layer1 + (output_layer1[i] - teacher_output_layer1[i]) ** 2
            if(i == TEACHER_SIZE-1):
                mean_squared_error_layer1 = mean_squared_error_layer1 / TEACHER_SIZE
                print('平均二条誤差')
                print(mean_squared_error_layer1)
        #パラメータ値の更新を行う
        #後件部実数値とモジュールの出力値の重みを更新する
        for j in range(NUMBER_OF_MODULAR_LAYER1):
            modular_weight_layer1[j] = modular_weight_layer1[j] + (LEARNING_WEIGHT * output_modular_layer1[i,j] * (teacher_output_layer1[i] - output_layer1[i]))
            for k in range(NUMBER_OF_FUZZY_RULE_OF_MODULAR):
                consequent_layer1[j,k] = consequent_layer1[j,k] + (LEARNING_CONSEQUENT * adaptability[i,j,k] / np.sum(adaptability[i,j])) * (teacher_output_layer1[i] - output_layer1[i])

平均二条誤差
0.08341639628559758
Wall time: 34.3 s


In [152]:
#2層目で用いる教師データとテストデータを作成する
#1層目で得られた結果を2層目の入力変数として扱う
teacher_input_layer2 = np.insert(teacher_input_layer1, NUMBER_OF_INPUT, output_layer1, axis=1)
teacher_output_layer2 = np.array(teacher_output_layer1)

In [153]:
%%time
#2層目の学習を行う
#平均二条誤差を設定する
mean_squared_error_layer2 = 0

#適合度・メンバシップ関数値を初期設定する
adaptability = np.ones((TEACHER_SIZE, NUMBER_OF_MODULAR_LAYER2, NUMBER_OF_FUZZY_RULE_OF_MODULAR))
membership_function = np.ones((TEACHER_SIZE, NUMBER_OF_MODULAR_LAYER2, NUMBER_OF_FUZZY_RULE_OF_MODULAR, 2))

for i in range(TEACHER_SIZE):
    for j in range(NUMBER_OF_MODULAR_LAYER2):
        for k in range(NUMBER_OF_FUZZY_RULE_OF_MODULAR):
            for l in range(2):
                #次にモジュラーの通し番号から用いる入力変数の属性を求める
                if l == 0:
                    #1つ目の入力変数属性に対して
                    count = 1
                    a = j
                    while True:
                        a = a - (NUMBER_OF_INPUT_LAYER2 - count)
                        count = count + 1
                        if a < 0:
                            m = count - 2
                            break
                else:
                    #2つ目の入力変数に対して
                    count = 1
                    a = j + 1
                    while True:
                        a = a - (NUMBER_OF_INPUT_LAYER2 - count)
                        count = count + 1
                        if a <= 0:
                            count = count - 1
                            n = NUMBER_OF_INPUT_LAYER2 - 1
                            m = n + a
                            break
                if(teacher_input_layer2[i,m] >= antecedent_center_layer2[j,k,l] - antecedent_broad_layer2[j,k,l]) and (teacher_input_layer2[i,m] <= antecedent_center_layer2[j,k,l]):
                    membership_function[i,j,k,l] = (teacher_input_layer2[i,m] - (antecedent_center_layer2[j,k,l] - antecedent_broad_layer2[j,k,l])) / antecedent_broad_layer2[j,k,l]
                elif(teacher_input_layer2[i,m] > antecedent_center_layer2[j,k,l]) and (teacher_input_layer2[i,m] <= antecedent_center_layer2[j,k,l] + antecedent_broad_layer2[j,k,l]):
                    membership_function[i,j,k,l] = -(teacher_input_layer2[i,m] - (antecedent_center_layer2[j,k,l] + antecedent_broad_layer2[j,k,l])) / antecedent_broad_layer2[j,k,l]
            #適合度を求める
            for l in range(2):
                adaptability[i,j,k] = adaptability[i,j,k] * membership_function[i,j,k,l]
#学習を行う
for time in range(TRAIN_TIME):
    #各データの予測結果を格納する
    output_layer2 = np.empty(TEACHER_SIZE)
    #各モジュールごとの出力値を格納する
    output_modular_layer2 = np.empty((TEACHER_SIZE, NUMBER_OF_MODULAR_LAYER2))
    for i in range(TEACHER_SIZE):
        for j in range(NUMBER_OF_MODULAR_LAYER2):
            for k in range(NUMBER_OF_FUZZY_RULE_OF_MODULAR):
                consequent_layer2[j,k] = consequent_part_layer2[j,k,0] * output_layer1[i] + consequent_part_layer2[j,k,1]
            #各データのモジュールごとの予測結果を求める
            output_modular_layer2[i,j] = np.sum(np.dot(adaptability[i,j], consequent_layer2[j])) / np.sum(adaptability[i,j])
        #各データの予測結果を求める
        output_layer2[i] = np.sum(np.dot(output_modular_layer2[i], modular_weight_layer2))
        if(time == TRAIN_TIME-1):
            mean_squared_error_layer2 = mean_squared_error_layer2 + (output_layer2[i] - teacher_output_layer2[i]) ** 2
            if(i == TEACHER_SIZE-1):
                mean_squared_error_layer2 = mean_squared_error_layer2 / TEACHER_SIZE
                print('平均二条誤差')
                print(mean_squared_error_layer2)
        #パラメータ値の更新を行う
        #後件部実数値とモジュールの出力値の重みを更新する
        for j in range(NUMBER_OF_MODULAR_LAYER2):
            modular_weight_layer2[j] = modular_weight_layer2[j] + (LEARNING_WEIGHT * output_modular_layer2[i,j] * (teacher_output_layer2[i] - output_layer2[i]))
            for k in range(NUMBER_OF_FUZZY_RULE_OF_MODULAR):
                for l in range(2):
                    if l == 0:
                        consequent_part_layer2[j,k,0] = consequent_part_layer2[j,k,0] + (LEARNING_CONSEQUENT * adaptability[i,j,k] / np.sum(adaptability[i,j])) * (teacher_output_layer2[i] - output_layer2[i]) * output_layer1[i]
                    else:
                        consequent_part_layer2[j,k,1] = consequent_part_layer2[j,k,1] + (LEARNING_CONSEQUENT * adaptability[i,j,k] / np.sum(adaptability[i,j])) * (teacher_output_layer2[i] - output_layer2[i])

平均二条誤差
0.07495395057104441
Wall time: 1min 43s


In [154]:
#3層目で用いる教師データとテストデータを作成する
teacher_input_layer3 = np.insert(teacher_input_layer2, NUMBER_OF_INPUT+1, output_layer2, axis=1)
teacher_output_layer3 = np.array(teacher_output_layer2)

In [155]:
%%time
#3層目の学習を行う
#平均二条誤差を設定する
mean_squared_error_layer3 = 0

#適合度・メンバシップ関数値を初期設定する
adaptability = np.ones((TEACHER_SIZE, NUMBER_OF_MODULAR_LAYER3, NUMBER_OF_FUZZY_RULE_OF_MODULAR))
membership_function = np.ones((TEACHER_SIZE, NUMBER_OF_MODULAR_LAYER3, NUMBER_OF_FUZZY_RULE_OF_MODULAR, 2))
for i in range(TEACHER_SIZE):
    for j in range(NUMBER_OF_MODULAR_LAYER3):
        for k in range(NUMBER_OF_FUZZY_RULE_OF_MODULAR):
            for l in range(2):
                #次にモジュラーの通し番号から用いる入力変数の属性を求める
                if l == 0:
                    #1つ目の入力変数属性に対して
                    count = 1
                    a = j
                    while True:
                        a = a - (NUMBER_OF_INPUT_LAYER3 - count)
                        count = count + 1
                        if a < 0:
                            m = count - 2
                            break
                else:
                    #2つ目の入力変数に対して
                    count = 1
                    a = j + 1
                    while True:
                        a = a - (NUMBER_OF_INPUT_LAYER3 - count)
                        count = count + 1
                        if a <= 0:
                            count = count - 1
                            n = NUMBER_OF_INPUT_LAYER3 - 1
                            m = n + a
                            break
                if(teacher_input_layer3[i,m] >= antecedent_center_layer3[j,k,l] - antecedent_broad_layer3[j,k,l]) and (teacher_input_layer3[i,m] <= antecedent_center_layer3[j,k,l]):
                    membership_function[i,j,k,l] = (teacher_input_layer3[i,m] - (antecedent_center_layer3[j,k,l] - antecedent_broad_layer3[j,k,l])) / antecedent_broad_layer3[j,k,l]
                elif(teacher_input_layer3[i,m] > antecedent_center_layer3[j,k,l]) and (teacher_input_layer3[i,m] <= antecedent_center_layer3[j,k,l] + antecedent_broad_layer3[j,k,l]):
                    membership_function[i,j,k,l] = -(teacher_input_layer3[i,m] - (antecedent_center_layer3[j,k,l] + antecedent_broad_layer3[j,k,l])) / antecedent_broad_layer3[j,k,l]
            #適合度を求める
            for l in range(2):
                adaptability[i,j,k] = adaptability[i,j,k] * membership_function[i,j,k,l]
            if adaptability[i,j,k] == 0:
                print('A')
#学習を行う
for time in range(TRAIN_TIME):
    #各データの予測結果を格納する
    output_layer3 = np.empty(TEACHER_SIZE)
    #各モジュールごとの出力値を格納する
    output_modular_layer3 = np.empty((TEACHER_SIZE, NUMBER_OF_MODULAR_LAYER3))
    for i in range(TEACHER_SIZE):
        for j in range(NUMBER_OF_MODULAR_LAYER3):
            for k in range(NUMBER_OF_FUZZY_RULE_OF_MODULAR):
                consequent_layer3[j,k] = consequent_part_layer3[j,k,0] * output_layer1[i] + consequent_part_layer3[j,k,1] * output_layer2[i] + consequent_part_layer3[j,k,2]
            #各データのモジュールごとの予測結果を求める
            output_modular_layer3[i,j] = np.sum(np.dot(adaptability[i,j], consequent_layer3[j])) / np.sum(adaptability[i,j])
        #各データの予測結果を求める
        output_layer3[i] = np.sum(np.dot(output_modular_layer3[i], modular_weight_layer3))
        if(time == TRAIN_TIME-1):
            mean_squared_error_layer3 = mean_squared_error_layer3 + (output_layer3[i] - teacher_output_layer3[i]) ** 2
            if(i == TEACHER_SIZE-1):
                mean_squared_error_layer3 = mean_squared_error_layer3 / TEACHER_SIZE
                print('平均二条誤差')
                print(mean_squared_error_layer3)
        #パラメータ値の更新を行う
        #後件部実数値とモジュールの出力値の重みを更新する
        for j in range(NUMBER_OF_MODULAR_LAYER3):
            modular_weight_layer3[j] = modular_weight_layer3[j] + (LEARNING_WEIGHT * output_modular_layer3[i,j] * (teacher_output_layer3[i] - output_layer3[i]))
            for k in range(NUMBER_OF_FUZZY_RULE_OF_MODULAR):
                for l in range(3):
                    if l == 0:
                        consequent_part_layer3[j,k,0] = consequent_part_layer3[j,k,0] + (LEARNING_CONSEQUENT * adaptability[i,j,k] / np.sum(adaptability[i,j])) * (teacher_output_layer3[i] - output_layer3[i]) * output_layer1[i]
                    elif l == 1:
                        consequent_part_layer3[j,k,1] = consequent_part_layer3[j,k,1] + (LEARNING_CONSEQUENT * adaptability[i,j,k] / np.sum(adaptability[i,j])) * (teacher_output_layer3[i] - output_layer3[i]) * output_layer2[i]
                    else:
                        consequent_part_layer3[j,k,2] = consequent_part_layer3[j,k,2] + (LEARNING_CONSEQUENT * adaptability[i,j,k] / np.sum(adaptability[i,j])) * (teacher_output_layer3[i] - output_layer3[i])

1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.5 0.5 0
A
1.0 0.0 1.0 0
A
1.0 0.0 1.0 0
A
1.0 0.0 

In [156]:
#次に1層目におけるテストを行う
